In [2]:
! pip install -q deeplake langchain
! pip install -q tensorflow_text
!pip install -q google-cloud-aiplatform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 580.3/580.3 kB 8.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3

In [7]:
!wget 'https://gist.githubusercontent.com/phillipj/4944029/raw/75ba2243dd5ec2875f629bf5d79f6c1e4b5a8b46/alice_in_wonderland.txt'  -O alice_in_wonderland.txt
!wget "https://raw.githubusercontent.com/amephraim/nlp/master/texts/J.%20K.%20Rowling%20-%20Harry%20Potter%201%20-%20Sorcerer's%20Stone.txt" -O Sorcerer_stone.txt

--2023-12-02 04:23:32--  https://gist.githubusercontent.com/phillipj/4944029/raw/75ba2243dd5ec2875f629bf5d79f6c1e4b5a8b46/alice_in_wonderland.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 148574 (145K) [text/plain]
Saving to: ‘alice_in_wonderland.txt’

alice_in_wonderland 100%[===================>] 145.09K  --.-KB/s    in 0.04s   

2023-12-02 04:23:32 (3.80 MB/s) - ‘alice_in_wonderland.txt’ saved [148574/148574]

--2023-12-02 04:23:32--  https://raw.githubusercontent.com/amephraim/nlp/master/texts/J.%20K.%20Rowling%20-%20Harry%20Potter%201%20-%20Sorcerer's%20Stone.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubuserconten

In [4]:
from langchain.embeddings import TensorflowHubEmbeddings
from langchain.vectorstores import DeepLake
url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual/3"
embed_model = TensorflowHubEmbeddings(model_url=url)

db = DeepLake(dataset_path='./vectordbs/deeplake/deeplake_default', \
                       embedding=embed_model)

In [8]:
def load_doc(filename,metatag):
  from langchain.document_loaders import TextLoader
  from langchain.text_splitter import CharacterTextSplitter

  loader = TextLoader(filename)
  documents = loader.load()
  text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
  docs = text_splitter.split_documents(documents)
  for d in docs:
    for key, value in metatag.items():
      d.metadata[key] = value
  return docs


In [9]:
alice_docs = load_doc('alice_in_wonderland.txt' ,{"author": "Lewis", "book": "Alice"})
stone_docs = load_doc('Sorcerer_stone.txt' ,{"author": "Rowling", "book": "Stone"})

In [10]:
lista_docs = db.add_documents(alice_docs)
lists_docs = db.add_documents(stone_docs)

Creating 178 embeddings in 1 batches of size 178:: 100%|██████████| 1/1 [00:09<00:00,  9.26s/it]


Dataset(path='./vectordbs/deeplake/deeplake_default', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
   text       text      (178, 1)     str     None   
 metadata     json      (178, 1)     str     None   
 embedding  embedding  (178, 512)  float32   None   
    id        text      (178, 1)     str     None   


Creating 512 embeddings in 2 batches of size 500:: 100%|██████████| 2/2 [00:18<00:00,  9.17s/it]

Dataset(path='./vectordbs/deeplake/deeplake_default', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
   text       text      (690, 1)     str     None   
 metadata     json      (690, 1)     str     None   
 embedding  embedding  (690, 512)  float32   None   
    id        text      (690, 1)     str     None   


In [11]:
query = "which game did alice and queen play"
docs = db.similarity_search(query)
print(docs[0].page_content)

Alice thought she might as well go back, and see how the game
was going on, as she heard the Queen's voice in the distance,
screaming with passion.  She had already heard her sentence three
of the players to be executed for having missed their turns, and
she did not like the look of things at all, as the game was in
such confusion that she never knew whether it was her turn or
not.  So she went in search of her hedgehog.

  The hedgehog was engaged in a fight with another hedgehog,
which seemed to Alice an excellent opportunity for croqueting one
of them with the other:  the only difficulty was, that her
flamingo was gone across to the other side of the garden, where
Alice could see it trying in a helpless sort of way to fly up
into a tree.


In [15]:
db.similarity_search(
    "What color was the Sorcerer stone?",
    filter={"metadata": {"author": "Rowling"}},
)

100%|██████████| 690/690 [00:00<00:00, 2482.93it/s]


[Document(page_content='She pushed the book toward them, and Harry and Ron read: The ancient\nstudy of alchemy is concerned with making the Sorcerer\'s Stone, a\nlegendary substance with astonishing powers. The stone will transform\nany metal into pure gold. It also produces the Elixir of Life, which\nwill make the drinker immortal.\n\nThere have been many reports of the Sorcerer\'s Stone over the centuries,\nbut the only Stone currently in existence belongs to Mr. Nicolas Flamel,\nthe noted alchemist and opera lover. Mr. Flamel, who celebrated his six\nhundred and sixty-fifth birthday last year, enjoys a quiet life in Devon\nwith his wife, Perenelle (six hundred and fifty-eight).\n\n"See?" said Hermione, when Harry and Ron had finished. "The dog must be\nguarding Flamel\'s Sorcerer\'s Stone! I bet he asked Dumbledore to keep it\nsafe for him, because they\'re friends and he knew someone was after it,\nthat\'s why he wanted the Stone moved out of Gringotts!"', metadata={'source': 'Sorc

In [16]:
%%writefile random01.txt
In a parallel dimension, gravity works in reverse, causing people to fall upward and land on clouds, where they have cloud picnics."

Scientists recently discovered a new species of talking trees that communicate in poetry and hold annual literature festivals in enchanted forests."

On a hidden island in the Pacific, Blue dolphins have developed a sophisticated form of underwater stand-up comedy, complete with dolphin laughter clubs."

In an imaginary desert, there's a mirage that actually materializes as a 24/7 dessert buffet, offering an infinite supply of calorie-free, delicious treats."

Archaeologists unearthed ancient laptops from a lost civilization, suggesting that they were far more technologically advanced than we are today, complete with prehistoric social media networks."

Writing random01.txt


In [17]:
db.add_documents(load_doc('random01.txt' ,{"author": "Random", "book": "Jobby"}))
db.similarity_search(
    "What color was the dolphins",
    filter={"metadata": {"author": "Random"}},
)



Creating 1 embeddings in 1 batches of size 1:: 100%|██████████| 1/1 [00:00<00:00, 13.81it/s]


Dataset(path='./vectordbs/deeplake/deeplake_default', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
   text       text      (691, 1)     str     None   
 metadata     json      (691, 1)     str     None   
 embedding  embedding  (691, 512)  float32   None   
    id        text      (691, 1)     str     None   


100%|██████████| 691/691 [00:00<00:00, 2660.54it/s]


[Document(page_content='In a parallel dimension, gravity works in reverse, causing people to fall upward and land on clouds, where they have cloud picnics."\n\nScientists recently discovered a new species of talking trees that communicate in poetry and hold annual literature festivals in enchanted forests."\n\nOn a hidden island in the Pacific, Blue dolphins have developed a sophisticated form of underwater stand-up comedy, complete with dolphin laughter clubs."\n\nIn an imaginary desert, there\'s a mirage that actually materializes as a 24/7 dessert buffet, offering an infinite supply of calorie-free, delicious treats."\n\nArchaeologists unearthed ancient laptops from a lost civilization, suggesting that they were far more technologically advanced than we are today, complete with prehistoric social media networks."', metadata={'source': 'random01.txt', 'author': 'Random', 'book': 'Jobby'})]

In [18]:
%%writefile random01.txt
In a parallel dimension, gravity works in reverse, causing people to fall upward and land on clouds, where they have cloud picnics."

Scientists recently discovered a new species of walking trees that communicate in poetry and hold annual literature festivals in enchanted forests."

On a hidden island in the Pacific, Pink dolphins have developed a sophisticated form of underwater stand-up comedy, complete with dolphin laughter clubs."

In an imaginary desert, there's a mirage that actually materializes as a 24/7 dessert buffet, offering an infinite supply of calorie-free, delicious treats."

Archaeologists unearthed ancient laptops from a lost civilization, suggesting that they were far more technologically advanced than we are today, complete with prehistoric social media networks."

Overwriting random01.txt


In [19]:
db.delete(
    filter={'metadata': {'author': 'Random'}}
)

100%|██████████| 691/691 [00:01<00:00, 689.42it/s]


True

In [20]:
db.add_documents(load_doc('random01.txt' ,{"author": "Random", "book": "Jobby"}))
db.similarity_search(
    "What color was the dolphins",
    filter={"metadata": {"author": "Random"}},
)

Creating 1 embeddings in 1 batches of size 1:: 100%|██████████| 1/1 [00:00<00:00, 17.08it/s]


Dataset(path='./vectordbs/deeplake/deeplake_default', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
   text       text      (691, 1)     str     None   
 metadata     json      (691, 1)     str     None   
 embedding  embedding  (691, 512)  float32   None   
    id        text      (691, 1)     str     None   


100%|██████████| 691/691 [00:00<00:00, 2622.25it/s]


[Document(page_content='In a parallel dimension, gravity works in reverse, causing people to fall upward and land on clouds, where they have cloud picnics."\n\nScientists recently discovered a new species of walking trees that communicate in poetry and hold annual literature festivals in enchanted forests."\n\nOn a hidden island in the Pacific, Pink dolphins have developed a sophisticated form of underwater stand-up comedy, complete with dolphin laughter clubs."\n\nIn an imaginary desert, there\'s a mirage that actually materializes as a 24/7 dessert buffet, offering an infinite supply of calorie-free, delicious treats."\n\nArchaeologists unearthed ancient laptops from a lost civilization, suggesting that they were far more technologically advanced than we are today, complete with prehistoric social media networks."', metadata={'source': 'random01.txt', 'author': 'Random', 'book': 'Jobby'})]

In [24]:
from google.colab import auth

auth.authenticate_user()

In [26]:
import vertexai
from langchain.llms import VertexAI

PROJECT_ID = 'zeta-yen-319702'
REGION = 'us-central1'
BUCKET = 'gs://zeta-yen-319702-temp/'

#os.environ['GOOGLE_APPLICATION_CREDENTIALS'] =

vertexai.init(
    project=PROJECT_ID,
    location=REGION,
    staging_bucket=BUCKET
)

llm = VertexAI(
    model_name="text-bison@001",
    max_output_tokens=256,
    temperature=0.8,
    top_p=0.8,
    top_k=5,
    verbose=True,
)

In [28]:
from langchain.chains import ConversationalRetrievalChain
qa = ConversationalRetrievalChain.from_llm(
    llm=VertexAI(temperature=0.5,top_p=0.8,top_k=0.8),
    retriever=db.as_retriever(k=1),
    return_source_documents=True,verbose=False
)


In [30]:
query = "Who was mad hatter?"
result = qa({"question": query,"chat_history" : ""})

result



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

She had not gone much farther before she came in sight of the
house of the March Hare:  she thought it must be the right house,
because the chimneys were shaped like ears and the roof was
thatched with fur.  It was so large a house, that she did not
like to go nearer till she had nibbled some more of the lefthand
bit of mushroom, and raised herself to about two feet high:  even
then she walked up towards it rather timidly, saying to herself
`Suppose it should be raving mad after all!  I almost wish I'd
gone to see the Hatter instead!'

                           CHAPTER VII

                         A Mad Tea-Party

`In THAT direction,' the Cat said, waving its right paw round,
`lives a Hatter:  and in THAT direction,' w

{'question': 'Who was mad hatter?',
 'chat_history': '',
 'answer': ' The hatter was mad. ',
 'source_documents': [Document(page_content="She had not gone much farther before she came in sight of the\nhouse of the March Hare:  she thought it must be the right house,\nbecause the chimneys were shaped like ears and the roof was\nthatched with fur.  It was so large a house, that she did not\nlike to go nearer till she had nibbled some more of the lefthand\nbit of mushroom, and raised herself to about two feet high:  even\nthen she walked up towards it rather timidly, saying to herself\n`Suppose it should be raving mad after all!  I almost wish I'd\ngone to see the Hatter instead!'\n\n                           CHAPTER VII\n\n                         A Mad Tea-Party", metadata={'source': 'alice_in_wonderland.txt'}),
  Document(page_content="`In THAT direction,' the Cat said, waving its right paw round,\n`lives a Hatter:  and in THAT direction,' waving the other paw,\n`lives a March Hare.  

In [31]:
# https://python.langchain.com/docs/integrations/vectorstores/activeloop_deeplake
import random

for d in docs:
    d.metadata["year"] = random.randint(2012, 2014)

db = DeepLake.from_documents(
    docs, embed_model, dataset_path="./my_deeplake/", overwrite=True
)

Creating 4 embeddings in 1 batches of size 4:: 100%|██████████| 1/1 [00:00<00:00,  7.96it/s]

Dataset(path='./my_deeplake/', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype     shape     dtype  compression
  -------    -------   -------   -------  ------- 
   text       text      (4, 1)     str     None   
 metadata     json      (4, 1)     str     None   
 embedding  embedding  (4, 512)  float32   None   
    id        text      (4, 1)     str     None   


In [32]:
db.similarity_search(
    "What did the president say about Ketanji Brown Jackson",
    filter={"metadata": {"year": 2013}},
)

100%|██████████| 4/4 [00:00<00:00, 706.44it/s]


[Document(page_content="All the time they were playing the Queen never left off\nquarrelling with the other players, and shouting `Off with his\nhead!' or `Off with her head!'  Those whom she sentenced were\ntaken into custody by the soldiers, who of course had to leave\noff being arches to do this, so that by the end of half an hour\nor so there were no arches left, and all the players, except the\nKing, the Queen, and Alice, were in custody and under sentence of\nexecution.\n\n  Then the Queen left off, quite out of breath, and said to\nAlice, `Have you seen the Mock Turtle yet?'\n\n  `No,' said Alice.  `I don't even know what a Mock Turtle is.'\n\n  `It's the thing Mock Turtle Soup is made from,' said the Queen.\n\n  `I never saw one, or heard of one,' said Alice.\n\n  `Come on, then,' said the Queen, `and he shall tell you his\nhistory,'", metadata={'source': 'alice_in_wonderland.txt', 'year': 2013}),
 Document(page_content="As they walked off together, Alice heard the King say in 

In [33]:
db.similarity_search(
    "What did the president say about Ketanji Brown Jackson?", distance_metric="cos"
)

[Document(page_content="Alice looked at the jury-box, and saw that, in her haste, she\nhad put the Lizard in head downwards, and the poor little thing\nwas waving its tail about in a melancholy way, being quite unable\nto move.  She soon got it out again, and put it right; `not that\nit signifies much,' she said to herself; `I should think it\nwould be QUITE as much use in the trial one way up as the other.'\n\n  As soon as the jury had a little recovered from the shock of\nbeing upset, and their slates and pencils had been found and\nhanded back to them, they set to work very diligently to write\nout a history of the accident, all except the Lizard, who seemed\ntoo much overcome to do anything but sit with its mouth open,\ngazing up into the roof of the court.\n\n  `What do you know about this business?' the King said to\nAlice.\n\n  `Nothing,' said Alice.\n\n  `Nothing WHATEVER?' persisted the King.\n\n  `Nothing whatever,' said Alice.", metadata={'source': 'alice_in_wonderland.txt',

In [34]:
db.max_marginal_relevance_search(
    "What did the president say about Ketanji Brown Jackson?"
)

[Document(page_content="Alice looked at the jury-box, and saw that, in her haste, she\nhad put the Lizard in head downwards, and the poor little thing\nwas waving its tail about in a melancholy way, being quite unable\nto move.  She soon got it out again, and put it right; `not that\nit signifies much,' she said to herself; `I should think it\nwould be QUITE as much use in the trial one way up as the other.'\n\n  As soon as the jury had a little recovered from the shock of\nbeing upset, and their slates and pencils had been found and\nhanded back to them, they set to work very diligently to write\nout a history of the accident, all except the Lizard, who seemed\ntoo much overcome to do anything but sit with its mouth open,\ngazing up into the roof of the court.\n\n  `What do you know about this business?' the King said to\nAlice.\n\n  `Nothing,' said Alice.\n\n  `Nothing WHATEVER?' persisted the King.\n\n  `Nothing whatever,' said Alice.", metadata={'source': 'alice_in_wonderland.txt',

In [1]:

db = DeepLake(dataset_path='./vectordbs/deeplake/deeplake_default', embedding=embed_model)

db.delete(
    filter={'metadata': {'key': 'value'}}
)

NameError: ignored